## Learning to chose the suit to lead on (pass) - 1nt - all pass
to add: treat the same hands with differents leads; check the dependence of 1nt force (!need to calculate points of declarer)

Script to get the leading hand and the lead from file "temp_final.DAT" which containts hands + leads

In [5]:
import parsing_temp_final
import numpy as np

In [2]:
filename = "temp_final.DAT"
vectors = parsing_temp_final.archiveTreat(filename)

Creating two arrays with hands and corresponding leads

In [3]:
leads = []
hands = []
length = len(vectors)
for i in range(length):
    leads.append(vectors[i][0])
    hands.append(vectors[i][1])
#S = 0, H = 1, D = 2, C = 3
suits_lead = []
for i in range(length):
    if leads[i][0] == 'S':
        suits_lead.append(0)
    if leads[i][0] == 'H':
        suits_lead.append(1)
    if leads[i][0] == 'D':
        suits_lead.append(2)
    if leads[i][0] == 'C':
        suits_lead.append(3)

Implementing the basic rule "lead the best suit", where "the best" means the longest (and major>minor)

In [6]:
number_of_cardes = np.zeros((length, 4), int)
for i in range(length):
    number_of_cardes[i][0] = sum(hands[i][0:13])
    number_of_cardes[i][1] = sum(hands[i][13:26])
    number_of_cardes[i][2] = sum(hands[i][26:39])
    number_of_cardes[i][3] = sum(hands[i][39:52])
best_suit = []
for i in range(length):
    best_suit.append(np.argmax(number_of_cardes[i]))

Calculate the percantage of applying the empiric rule in the given data

In [7]:
a = np.array(best_suit)
b = np.array(suits_lead)
print 1.0*np.count_nonzero(a==b)/length

0.738398544131


Making a classifier

In [8]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

Learning and checking a OneVsRestClassifier classifier on train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(hands, suits_lead, test_size=0.20, random_state=0)
clf = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train)
clf.score(X_test, y_test)

0.6886363636363636

Learning and checking a SVM classifier on train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(hands, suits_lead, test_size=0.2, random_state=0)
clf_svm = SVC()
clf_svm.fit(X_train, y_train)
clf_svm.score(X_test, y_test)

0.67500000000000004

Increasing number of boards by swipping majors and minors

In [15]:
#exchange majors; exchange minors; exchange both
for i in range(length):
    
    #exchange majors
    new_hand = np.zeros(52)
    new_hand[:13], new_hand[13:26], new_hand[26:52] = hands[i][13:26], hands[i][0:13], hands[i][26:52]
    hands.append(new_hand)
    if suits_lead[i] == 0:
        suits_lead.append(1)
    if suits_lead[i] == 1:
        suits_lead.append(0)
    if suits_lead[i] == 2:
        suits_lead.append(2)
    if suits_lead[i] == 3:
        suits_lead.append(3)
        
    #exchange minors
    new_hand = np.zeros(52)
    new_hand[:26], new_hand[26:39], new_hand[39:52] = hands[i][0:26], hands[i][39:52], hands[i][26:39]
    hands.append(new_hand)
    if suits_lead[i] == 0:
        suits_lead.append(0)
    if suits_lead[i] == 1:
        suits_lead.append(1)
    if suits_lead[i] == 2:
        suits_lead.append(3)
    if suits_lead[i] == 3:
        suits_lead.append(2)
        
    #exchange both
    new_hand = np.zeros(52)
    new_hand[:13], new_hand[13:26], new_hand[26:39], new_hand[39:52] = hands[i][13:26], hands[i][0:13], hands[i][39:52], hands[i][26:39]
    hands.append(new_hand)
    if suits_lead[i] == 0:
        suits_lead.append(1)
    if suits_lead[i] == 1:
        suits_lead.append(0)
    if suits_lead[i] == 2:
        suits_lead.append(3)
    if suits_lead[i] == 3:
        suits_lead.append(2)

Learning and checking bigger dataset on a OneVsRestClassifier classifier on train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(hands, suits_lead, test_size=0.2, random_state=0)
clf = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train)
clf.score(X_test, y_test)

0.79306424104604889